## Welcome to your notebook.


#### Run this cell to connect to your GIS and get started:

In [ ]:
from arcgis.gis import GIS
gis = GIS("home")

# Join tables based on Email Address

In [ ]:
# publish tables that christina gave me - done
# read in the tables - done
# create email_lower column and populate with lower case version of the email 
# trim whitespace from both emails so that the join is successful
# join on this EMAIL UPPER column
# update 4 fields in PII services table based on the data in the incoming table

In [ ]:
import pandas as pd
from pandas.testing import assert_frame_equal

#### Read in Hosted Tables as Dataframes

In [ ]:
# Add pii services item
pii_services_item = gis.content.get("c2e4b1b7254047a3ba970bddda528f7d") # update this item to point to the PII Services itemid
pii_services_item

In [ ]:
# convert hosted table layer as a data frame
# https://community.esri.com/t5/arcgis-online-questions/access-non-spatial-hosted-table-with-python/m-p/1255062#M50136
pii_services_df = pii_services_item.tables[0].query(as_df=True)
pii_services_df

In [ ]:
# Add dm circuit item
dm_circuit_item = gis.content.get("e3226cdd7d5c4cd794f205b472d012d7")
dm_circuit_item

In [ ]:
# convert hosted table layer as a data frame
# https://community.esri.com/t5/arcgis-online-questions/access-non-spatial-hosted-table-with-python/m-p/1255062#M50136
dm_circuit_df= dm_circuit_item.tables[0].query(as_df=True)
dm_circuit_df

In [ ]:
# check the column names
# we require keeping the object id of the PII Services table as it will be used to update records accordingly
print(pii_services_df.columns)
print(dm_circuit_df.columns)

##### Create email lower column

In [ ]:
# create email_lower field and remove white space from field
pii_services_email_lower_df = pii_services_df
pii_services_email_lower_df['email_lower'] = pii_services_email_lower_df['DME_Email_Address'].apply(lambda x: x.lower() if isinstance(x, str) else x).str.strip()
pii_services_email_lower_df

In [ ]:
pii_services_email_lower_df.columns

In [ ]:
# create email_lower field and remove white space from field
# drop objectid column
dm_circuit_email_lower_df = dm_circuit_df
dm_circuit_email_lower_df['email_lower'] = dm_circuit_email_lower_df['Email'].apply(lambda x: x.lower() if isinstance(x, str) else x).str.strip()
dm_circuit_email_lower_df = dm_circuit_email_lower_df.drop(['ObjectId'], axis=1)
dm_circuit_email_lower_df

In [ ]:
dm_circuit_email_lower_df.columns

### Join PII Services table and Incoming table on lower case email column (email_lower)

In [ ]:
# Join the dataframes on the 'ID' column
merged_df = pd.merge(pii_services_email_lower_df, dm_circuit_email_lower_df, on='email_lower', how='inner')
merged_df

In [ ]:
# Update DME_Total_Listing_Reviewed field with value from joined table if present
merged_df.columns
merged_df['DME_Total_Listing_Reviewed']

In [ ]:
merged_df['DME_Total_Listing_Removed']

In [ ]:
merged_df['DME_Total_Searches']

In [ ]:
merged_df['DME_Total_PII_Removed']

### Update PII Services Dataframe with values from join table

In [ ]:
merged_df['DME_Total_Listing_Reviewed'] = merged_df.apply(lambda row: row['Total_Listings_Reviewed'] if row['Total_Listings_Reviewed'] else row['DME_Total_Listing_Reviewed'], axis=1)
merged_df['DME_Total_Listing_Removed'] = merged_df.apply(lambda row: row['Total_Listings_Removed'] if row['Total_Listings_Removed'] else row['DME_Total_Listing_Removed'], axis=1)
merged_df['DME_Total_Searches'] = merged_df.apply(lambda row: row['Total_Searches'] if row['Total_Searches'] else row['DME_Total_Searches'], axis=1)
merged_df['DME_Total_PII_Removed'] = merged_df.apply(lambda row: row['Total_PII_Removed'] if row['Total_PII_Removed'] else row['DME_Total_Listing_Reviewed'], axis=1)
merged_df

In [ ]:
merged_df['DME_Total_Listing_Reviewed']

In [ ]:
merged_df['DME_Total_Listing_Removed']

In [ ]:
merged_df['DME_Total_Searches']

In [ ]:
merged_df['DME_Total_PII_Removed']

### Create updates Dataframe with only the columns that need to be updated

In [ ]:
updates_df = merged_df[['ObjectId', 'DME_Email_Address', 'DME_Total_Listing_Reviewed', 'DME_Total_Listing_Removed', 'DME_Total_Searches', 'DME_Total_PII_Removed']].copy()
updates_df

### Update the PII Services Hosted Table Layer

In [ ]:
def batch_edits(feature_list, batch_size):

    for i in range(0, len(feature_list), batch_size):
        yield feature_list[i:i + batch_size]

def append_service(gis, input_sdf, input_service_itemid, input_service_layerid):

    input_layer_fs = input_sdf.spatial.to_featureset()

    # Chunk edits into lists
    batch_size = 25
    update_sets = list(batch_edits(input_layer_fs.features, batch_size))

    # Get Hosted Table Layer Service to Update
    input_service = gis.content.get(input_service_itemid).tables[input_service_layerid]
    
    print(input_service)

    # Append Feature Service
    for edits in update_sets:
        res = input_service.edit_features(updates=edits, rollback_on_failure=False)['updateResults']
        for obj in res:
            if(obj['success']):
                print("Successfully updated ObjectId: " + str(obj['objectId']))
            else:
                print("ObjectId " + str(obj['objectId']) + " not updated!" )

In [ ]:
# update the PII Services Hosted table using the updated_df
if not updates_df.empty:
    print("Dataframe is not empty.")
    append_service(gis, updates_df, "e3226cdd7d5c4cd794f205b472d012d7", 0) # update the itemid and the hosted table index (0 - in my example to the left)
    print("Updated Records successfully!")
else:
    print("Failed to update records in PII Services Hosted Table.")

----------------------------------------------------